In [ ]:
%matplotlib inline
import pynbody
from pynbody.analysis import profile
import matplotlib.pylab as plt
import numpy as np
import astropy.units as u
import os

In [ ]:
from cog import compute_cog
from snap_io import load_moria_sim_and_kicked, load_moria, load_kicked, load_sim

In [ ]:
def get_cog(sim_dir):
    sim = load_sim(sim_dir)
    print("Loaded {} snaps of sim: {}".format(len(sim), sim_dir))
    times, cog = compute_cog(sim, save_cache=False, verbose=False);
    return times, cog

def get_cog_kicked(sim_number):
    kicked_sim = load_kicked(sim_number)
    print("Loaded {} snaps of sim{}".format(len(kicked_sim), sim_number))
    times, cog = compute_cog(kicked_sim, cache_file='cog{}_kicked.npz'.format(sim_number), verbose=False);
    return times, cog

In [ ]:
def plot_cog_path(cog_kicked, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_xlabel("x (kpc)")
    ax.set_ylabel("y (kpc)")
    ax.scatter(*cog_kicked[:2], marker='o')
    ax.plot((0,0), '')
    c1, c2 = plt.Circle((0,0), radius=1000, fill=False), plt.Circle((0,0), radius=200, fill=False)
    ax.add_artist(c1)
    ax.add_artist(c2)
    ax.axis('equal')
    ax.grid()
    r = np.sqrt(cog_kicked[0,:]**2+cog_kicked[1,:]**2+cog_kicked[2,:]**2)
    r[:-10].min(), r.max()

In [ ]:
sim_list = (62002, 68002, 71002, 60003)

In [ ]:
times_list = list()
cog_list = list()
for sim in sim_list:
    times, cog = get_cog_kicked(sim)
    times_list.append(times)
    cog_list.append(cog)
import gc
gc.collect()

In [ ]:
len(cog_list)

In [ ]:
nplots = len(sim_list)
fig, axes = plt.subplots(nplots, figsize=(8,8*nplots))
for cog, sim, ax in zip(cog_list, sim_list, axes):
    plot_cog_path(cog, ax)
    ax.set_title("sim{}".format(sim))

In [ ]:
times, cog = get_cog("/home/michele/sim/nfw_negative/out")

In [ ]:
fig, ax = plt.subplots(1, figsize=(8,8))
plot_cog_path(cog, ax)
ax.set_title("{}".format("nfw_negative"))